In [116]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from pathlib import *
import time
import pandas as pd

In [117]:
url_poems = 'https://www.poemes.co{}'
theme_extension = '/themes.html'

req = requests.get(url_poems.format(theme_extension))
soup = BeautifulSoup(req.text, "lxml")

In [118]:
table = soup.find('table',{'class','taxonomy-vtn-terms'})
table

<table class="taxonomy-vtn-terms" id="poetes_terms">
<tbody>
<tr class="odd"><td style="width:50%"><div class="item-list"><h3>A</h3><ul><li class="first"><a href="/poemes-sur-l-absence.html">Absence</a> (2)</li>
<li><a href="/poemes-d-adieu.html">Adieu</a> (5)</li>
<li><a href="/amitie.html">Amitié</a> (180)</li>
<li><a href="/poemesdamour.html">Amour</a> (630)</li>
<li><a href="/ange.html">Ange</a> (37)</li>
<li><a href="/angoisse.html-1">Angoisse</a> (17)</li>
<li><a href="/animaux.html">Animaux</a> (312)</li>
<li><a href="/poemes-d-anniversaire.html">Anniversaire</a> (3)</li>
<li><a href="/art.html">Art</a> (83)</li>
<li><a href="/poesie-poemes-saison-automne.html">Automne</a> (62)</li>
<li><a href="/avarice.html">Avarice</a> (3)</li>
<li class="last"><a href="/avenir.html-1">Avenir</a> (21)</li>
</ul></div><div class="item-list"><h3>B</h3><ul><li class="first"><a href="/beaute.html">Beauté</a> (175)</li>
<li><a href="/poemes-belles-femmes.html">Belles Femmes</a> (13)</li>
<li><a hr

In [119]:
links = []
themes = []

for th in table.find('tbody').find_all('tr'):    
    for a in th.find_all('a'):
        if a != None:
            themes.append(a.get_text())
            links.append(url_poems.format(a.get("href")))

driver_path = Path.cwd() / '../chromedriver'
driver = webdriver.Chrome(executable_path=driver_path.as_posix())

In [120]:
list_poems_content = []
list_author = []
list_poems_title = []
list_theme = []
    
for l in links:
    
    #picking current poems thems to create a parallele list
    index = links.index(l)
    current_theme = themes[index]
    
    list_poems_link = []
    #scrapping all poems for a specific theme
    driver.get(l)
    poems = driver.find_element_by_class_name('category-col-left').find_elements_by_class_name('menu')
    for p in poems:
        poems_link = p.find_element_by_tag_name('a').get_attribute('href')
        poems_title = p.find_element_by_tag_name('a').text
        
        list_poems_title.append(poems_title)
        list_poems_link.append(poems_link)

    #iterate on each poems link to scrapp its content and author
    for pl in list_poems_link:
        list_theme.append(current_theme)
        driver.get(pl)
        time.sleep(1/2)

        #get the poetry content
        poems_content = driver.find_element_by_class_name('entry-description')\
                              .find_elements_by_tag_name('p')

        sub_list_poems_content = []
                
        for poem_pieces in poems_content:
            parent = poem_pieces.find_element_by_xpath('./..').get_attribute("class")

        #condition to check if the element is really part of the poem or just comments about it
            if parent == 'entry-description':
                sub_list_poems_content.append(poem_pieces.text)

        #merge all element of the list into one string : the poem
        full_poem_content = ''.join(sub_list_poems_content)
        #Add the poem to the list of poems
        list_poems_content.append(full_poem_content)
                
        #get the author name                   
        author = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/article/header/div/span')
        list_author.append(author.text[4:])        
                                               
    link_position = links.index(l) + 1
    link_list_size = len(links)
    print('{} link done out of {}'.format(link_position,link_list_size))

driver.quit()

1 link done out of 125
2 link done out of 125
3 link done out of 125
4 link done out of 125
5 link done out of 125
6 link done out of 125
7 link done out of 125
8 link done out of 125
9 link done out of 125
10 link done out of 125
11 link done out of 125
12 link done out of 125
13 link done out of 125
14 link done out of 125
15 link done out of 125
16 link done out of 125
17 link done out of 125
18 link done out of 125
19 link done out of 125
20 link done out of 125
21 link done out of 125
22 link done out of 125
23 link done out of 125
24 link done out of 125
25 link done out of 125
26 link done out of 125
27 link done out of 125
28 link done out of 125
29 link done out of 125
30 link done out of 125
31 link done out of 125
32 link done out of 125
33 link done out of 125
34 link done out of 125
35 link done out of 125
36 link done out of 125
37 link done out of 125
38 link done out of 125
39 link done out of 125
40 link done out of 125
41 link done out of 125
42 link done out of 125
4

In [121]:
#store the data into a dataframe

df = pd.DataFrame({'Theme':list_theme,
                   'Title':list_poems_title,
                   'Author':list_author,
                   'Content':list_poems_content
                  }
)
df.head()

,Theme,Title,Author,Content
0,Absence,Est-ce le Vent...?,PHILIPPE SOUPAULT,Est-ce le vent qui m’apporte tout à coup ces n...
1,Absence,Sous les Arbres Mauves,PHILIPPE SOUPAULT,Sous les arbres mauves\nune nuit mauvaise\nj’a...
2,Adieu,Au Revoir,RALPH WALDO EMERSON,"Au revoir, monde fier! je rentre à la maison:\..."
3,Adieu,Ceci est mon Testament,ROSEMONDE GÉRARD,"Je vous laisse, Ami cher, cette frivole estamp..."
4,Adieu,Les Adieux,SAPPHO,Atthis n'est point sur ses pas retournée.\nVra...


In [122]:
#saving dataframe as csv
df.to_csv('../Data/poems_dataset')